In [ ]:
import pandas as pd
import numpy as np
import glob
from sqlalchemy import create_engine

def extract(file_path):
  csv_files = glob.glob(file_path)
  data_frames = [pd.read_csv(file) for file in csv_files]
  # Combine all data frames into one
  df = pd.concat(data_frames, ignore_index=True)
  return df

def transform (df):
  #handling missing values
  df.dropna(inplace=True)
  #handling duplicates
  df.drop_duplicates(subset=['Store_id','date','product_id'],inplace=True)
  #handling inconsistent formats
  df['date'] = pd.to_datetime(df['date'])
  df['store_id'] = df['store_id'].astype('str')
  df['product_id'] = df['product_id'].astype('str')
  df['quantity_sold'] = df['quantity_sold'].astype('int')
  df['quantity_in_stock'] = df['quantity_in_stock'].astype('int')
  df['revenue'] = df['revenue'].astype('float')
  return df


#Load cleaned data to sql database with appropriate indexing
def load(df):
  engine = create_engine('sqlite:///tophome.db')
  df.to_sql('sales',con=engine,if_exists='append',index=False)
  # Create indexes for efficient querying
  with engine.connect() as conn:
    conn.execute('CREATE INDEX idx_store_date ON sales (store_id, date)')
    conn.execute('CREATE INDEX idx_product ON sales (product_id)')

data = extract('*.csv')
if not data.empty:
    sales = transform(data)
    load(sales)
else:
    print("No data to process.")